# AlphaGIS Home Assignment
Georg Simmul

### Initial Setup
ArcGis has to be installed separately to Google Colab. This cell should be uncommented if running code in Google Colab, otherwise ignore.

In [148]:
#!sudo apt-get install libkrb5-dev
#!pip install arcgis

# To check if package is intalled, run:
#!pip list

### Installing packages

As ArcGIS package isn't officially supported in Google Colab the map widget can't be displayed. So in this case a different package `folium` (Leaflet) is used to display and save the map view.

In [139]:
# Importing necessary packages
from arcgis.gis import GIS
from arcgis.geocoding import geocode
import folium

### Geocoding and calculating distance

In [141]:
# Creating the GIS object
gis = GIS()

In [145]:
# Define the addresses used
office_adr = "Veerenni 40a-63, 10138 Tallinn"
customer_adr = [
    "Kopli OÜ, Sepa 24, 11712 Tallinn",
    "Tehas OÜ, Marati 5, 11713 Tallinn",
    "Stroomi OÜ, Pelguranna 51, Tallinn",
    "Hippo OÜ, Merimetsa tee 1, 10614 Tallinn",
    "Löwen OÜ, Mustamäe tee 59a, 10621 Tallinn",
    "Järvi OÜ, Tammiku tee 15, Jüri, 75305 Harju maakond",
    "Nurme OÜ, Nurme 18, Saku, 75501 Harju maakond",
    "Kose OÜ, Pikk 39, Kose, 75101 Harju maakond",
    "Lume OÜ, Lumemarja tee 15, Haabneeme, 74001 Harju maakond",
    "Waldi OÜ, F. R. Kreutzwaldi 9, 10126 Tallinn"
]

In [143]:
# Geocode the address of the office, get the closest match
office_loc = geocode(office_adr, max_locations=1)[0]

In [ ]:
# Define a function to merge two dictionaries
def add_dict(dict1,dict2):
  dict1.update(dict2)
  return dict1

In [ ]:
# Extract the coordinates of the office location
office_coord = [office_loc["location"]["x"], office_loc["location"]["y"]]

In [ ]:
# Geocode each customer address, take only the best match, add the input address
# Providing the office coordinates allows the method to calculate distance to office as well
customer_loc = [add_dict(geocode(adr, max_locations=1, location=office_coord)[0], {'original_address': adr}) for adr in customer_adr]

In [146]:
# Print distance of each address to the office location in meters
[{loc["original_address"]: loc["attributes"]["Distance"]} for loc in customer_loc]

[{'Kopli OÜ, Sepa 24, 11712 Tallinn': 6570.252708304949},
 {'Tehas OÜ, Marati 5, 11713 Tallinn': 6209.810360535172},
 {'Stroomi OÜ, Pelguranna 51, Tallinn': 4807.185261954121},
 {'Hippo OÜ, Merimetsa tee 1, 10614 Tallinn': 3094.3220287463764},
 {'Löwen OÜ, Mustamäe tee 59a, 10621 Tallinn': 3189.598983379351},
 {'Järvi OÜ, Tammiku tee 15, Jüri, 75305 Harju maakond': 10757.978340043403},
 {'Nurme OÜ, Nurme 18, Saku, 75501 Harju maakond': 13760.117503466852},
 {'Kose OÜ, Pikk 39, Kose, 75101 Harju maakond': 35275.648499936695},
 {'Lume OÜ, Lumemarja tee 15, Haabneeme, 74001 Harju maakond': 11606.33942671737},
 {'Waldi OÜ, F. R. Kreutzwaldi 9, 10126 Tallinn': 2335.6533735881007}]

### Generate and save the map view

As previously stated, using the `folium` map for this.

In [ ]:
# Create a folium map centered at the coordinates of the office
map = folium.Map(office_coord[::-1])

# Add a marker for the office location with a popup displaying its address
folium.Marker(location=office_coord[::-1],
              popup=folium.Popup(folium.IFrame("Name: AlphaGIS<br>Address: {}".format(office_adr)), min_width=300, max_width=3000),
              icon=folium.Icon(color="red", icon="building", prefix="fa")
              ).add_to(map)

# Iterate through customer locations
for loc in customer_loc:
  loc_coord = [loc["location"]["x"], loc["location"]["y"]]
  attributes = loc["attributes"]

  # Create popup for customer location marker
  popup = folium.Popup(folium.IFrame("Name: {}<br>Distance: {:.2f} km<br>Address: {}".format(attributes["ExInfo"],attributes["Distance"]/1000,loc["address"] if loc["address"] else "[missing]")),
                        min_width=300, max_width=3000)

  # Add marker for customer location
  folium.Marker(location=loc_coord[::-1], popup=popup, icon=folium.Icon(color="blue", icon="c", prefix='fa')).add_to(map)

  # Add polyline from office to customer location
  folium.PolyLine([office_coord[::-1], loc_coord[::-1]], weight=4, opacity=0.5, tooltip="{:.2f} km".format(attributes["Distance"]/1000)).add_to(map)

In [147]:
map # Display the folium map

In [ ]:
# To save the folium map as an HTML file, run:
#map.save("home_assignment_map.html")